# **Crisis NLP Analysis with Sieves**

<img src="assets/sieves_logo.png" width=200px height=200px />


## Table of Contents
1. [Imports](#imports)
2. [Dataset Loading](#dataset-loading)
3. [Data Exploration](#data-exploration)  
4. [Creating Sieves Doc Objects](#creating-sieves-doc-objects)
5. [Defining Models](#defining-models)
6. [Language Detection](#language-detection)
7. [Main Pipeline](#main-pipeline)
8. [Evaluating Results](#evaluating-results)

### Imports

Dependencies:
ipykernel
pandas
openai
outlines
sieves


In [8]:
import pandas as pd
import os
import openai
import outlines
from sieves import Pipeline, tasks, Doc
from sieves.tasks.predictive.classification import FewshotExampleMultiLabel
from pydantic import BaseModel, Field
from typing import Literal
import random

# Number of rows per crisis type to subset to
n_rows = 100

# Subset of tweets
n_tweets = 100


### Dataset Loading

Download the Dataset. If you are unable to run the following code block you can download manually the dataset with this url:

https://crisisnlp.qcri.org/data/lrec2016/labeled_cf/CrisisNLP_labeled_data_crowdflower_v2.zip

Then you can place is in a directory with path /examples/use-case/CrisisNLP

In [ ]:
import urllib.request
import zipfile
from pathlib import Path
import shutil

url = "https://crisisnlp.qcri.org/data/lrec2016/labeled_cf/CrisisNLP_labeled_data_crowdflower_v2.zip"
zip_path = Path("CrisisNLP_dataset.zip")
data_dir = Path("CrisisNLP")

if not zip_path.exists():
    urllib.request.urlretrieve(url, str(zip_path))

with zipfile.ZipFile(str(zip_path), 'r') as z:
    z.extractall(".")

extracted = Path("CrisisNLP_labeled_data_crowdflower")
if extracted.exists():
    data_dir.mkdir(exist_ok=True)
    for item in extracted.iterdir():
        target = data_dir / item.name
        if not target.exists():
            item.rename(target)
    shutil.rmtree(extracted)


Load the data

In [9]:
data: pd.DataFrame = pd.DataFrame()

print("Available Crisis Types:\n")
for file in os.listdir("CrisisNLP"):
    if os.path.isdir(os.path.join("CrisisNLP", file)):
        print(f"- {file}")
        for file_ in os.listdir(os.path.join("CrisisNLP", file)):
            if file_.endswith(".tsv"):
                data_ = pd.read_csv(os.path.join("CrisisNLP", file, file_), sep="\t")
                
                if n_rows is not None:
                    # Subset to n_rows rows for testing
                    data_ = data_[0:n_rows]
                
                data_["location"] = file.split("_")[0]
                data_["crisis_type"] = file.split("_")[1]
                data = pd.concat([data, data_])



Available Crisis Types:

- India_Floods
- California_Earthquake
- Pakistan_Earthquake
- Mexico_Hurricane
- Senegal_Ebola
- Vanuatu_Cyclone
- Nepal_Earthquake
- Philippines_Typhoon


### Data Exploration

In [10]:
data.tail()

,tweet_id,tweet_text,label,location,crisis_type
95,'541911446318112768',Please leave the Philippine Area of Responsibi...,caution_and_advice,Philippines,Typhoon
96,'541256603459407872',RT @JenArellanooo: “@NBCLA: Half a million Fil...,displaced_people_and_evacuations,Philippines,Typhoon
97,'540558654064570368',RT @WilliamHuizinga: Tyfoon #Hagupit RT @eumet...,other_useful_information,Philippines,Typhoon
98,'541138632032010241',Typhoon Hagupit http://t.co/pF13CVk9CV from #C...,other_useful_information,Philippines,Typhoon
99,'541500939451240448',RT @PhilippineStar: PHOTO: #RubyPH / #Hagupit ...,other_useful_information,Philippines,Typhoon


In [11]:
data["location"].value_counts()


location
India          100
California     100
Pakistan       100
Mexico         100
Senegal        100
Vanuatu        100
Nepal          100
Philippines    100
Name: count, dtype: int64

In [12]:
print("Number of rows:", len(data))
print("Number of columns:", len(data.columns))
print("Column names:", data.columns.tolist())

print("\nSummary statistics:")
display(data.describe(include='all'))


Number of rows: 800
Number of columns: 5
Column names: ['tweet_id', 'tweet_text', 'label', 'location', 'crisis_type']

Summary statistics:


,tweet_id,tweet_text,label,location,crisis_type
count,800,800,800,800,800
unique,800,800,15,8,6
top,'501222299685888000',RT @lilid426: Everyone has all these intense r...,other_useful_information,India,Earthquake
freq,1,1,211,100,300


### Creating Sieves Doc Objects


In [13]:
# Convert tweets to Sieves Doc objects
def create_sieves_docs(df):
    """Create Sieves Doc objects from DataFrame"""
    docs = []
    for idx, row in df.iterrows():
        doc = Doc(
            uri=f"tweet_{row['tweet_id']}",
            text=row['tweet_text']
        )
        # Add metadata
        doc.metadata = {
            'tweet_id': row['tweet_id'],
            'gold_label': row['label'],
            'location': row['location'],
            'crisis_type': row['crisis_type']
        }
        docs.append(doc)
    return docs

# Create docs from English tweets
docs = create_sieves_docs(data)
print(f"Created {len(docs)} Sieves Doc objects")
print(f"\nSample doc:")
print(f"Tweet: {docs[0].text[:100]}...")
print(f"Metadata: {docs[0].metadata}")


Created 800 Sieves Doc objects

Sample doc:
Tweet: RT @lilid426: Everyone has all these intense relationship goals when my goal is just getting in a re...
Metadata: {'tweet_id': "'501222299685888000'", 'gold_label': 'not_related_or_irrelevant', 'location': 'India', 'crisis_type': 'Floods'}


### Defining models

In [14]:
# Using OpenAI models via outlines
model = outlines.from_openai(
    openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"]),
    model_name="gpt-4o"
)
small_model = outlines.from_openai(
    openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"]),
    model_name="gpt-4o-mini"
)

Subset of docs

In [15]:
if n_tweets is not None:
    test_docs = random.sample(docs,n_tweets)
else:
    test_docs = docs

### Language Detection

Define the language detection few shot examples, task and pipeline

In [16]:
from sieves.tasks.predictive.classification import FewshotExampleMultiLabel

fewshot_examples = [
    FewshotExampleMultiLabel(
        text="Help! There's a major earthquake in California. Everyone stay safe!",
        reasoning="This text is clearly written in English with standard English words, grammar, and sentence structure.",
        confidence_per_label={"english": 1.0, "other": 0.0}
    ),
    FewshotExampleMultiLabel(
        text="¡Ayuda! Hay un terremoto en México. ¡Manténganse seguros!",
        reasoning="This text is written in Spanish, using Spanish words like 'Ayuda' (help) and 'terremoto' (earthquake), and Spanish grammar structures.",
        confidence_per_label={"english": 0.0, "other": 1.0}
    ),
    FewshotExampleMultiLabel(
        text="Praying for the victims of the flood in Nepal. #PrayForNepal",
        reasoning="This is an English tweet with standard English vocabulary, grammar, and hashtag format commonly used on Twitter.",
        confidence_per_label={"english": 0.95, "other": 0.05}
    ),
    FewshotExampleMultiLabel(
        text="Bonjour! Comment allez-vous? Je parle français.",
        reasoning="This text contains French words and phrases. While it has some basic structure, it's clearly not English.",
        confidence_per_label={"english": 0.1, "other": 0.9}
    ),
    FewshotExampleMultiLabel(
        text="RT @user: Just saw the news about the hurricane. Hope everyone is okay!",
        reasoning="This is an English tweet with typical Twitter retweet format (RT), English words, and standard English sentence structure.",
        confidence_per_label={"english": 0.98, "other": 0.02}
    ),
    FewshotExampleMultiLabel(
        text="Hello world! This is a test message with some English words but also 你好世界",
        reasoning="This text contains both English and Chinese characters. It's mixed language, so while English is present, it's not entirely English.",
        confidence_per_label={"english": 0.6, "other": 0.7}
    )
]

Define the task

In [17]:
language_detector = tasks.Classification(
        task_id="language_detector",
        labels=["english", "other"],
        model=small_model,
        label_descriptions={"english": "The tweet is in English.", "other": "The tweet is in a language other than English."},
        prompt_instructions="""
        You are a helpful assistant that determines the language of a given text. If the text is in English, return 'english'. Otherwise, return 'other'.
        """,
        fewshot_examples=fewshot_examples,
    )

language_detection = Pipeline([language_detector])
print(f"Number of tasks: {len(language_detection.tasks)}")

Number of tasks: 1


Run the language detection pipeline

In [18]:
try:
    language_detection_results = list(language_detection(test_docs))
    print("Language detection pipeline run successfully!")
except Exception as e:
    print(f"Error running language detection: {e}")

Running pipeline: 100%|██████████| 100/100 [02:56<00:00,  1.76s/it] 

Language detection pipeline run successfully!


Show results

In [19]:
language_detection_results[:10]

[Doc(meta={}, results={'language_detector': [('english', 1.0), ('other', 0.0)]}, uri="tweet_'540375974136004611'", text='RT @BuzzFeedStorm: Super Typhoon Hagupit strengthens with 178 mph max winds as storm tracks toward Philippines, JTWC reports http://t.co/ay…', chunks=['RT @BuzzFeedStorm: Super Typhoon Hagupit strengthens with 178 mph max winds as storm tracks toward Philippines, JTWC reports http://t.co/ay…'], id=None, images=None),
 Doc(meta={}, results={'language_detector': [('english', 1.0), ('other', 0.0)]}, uri="tweet_'575858180917342208'", text='RT @disasteraware: #CYCLONE Alert-Update: Tropical Cyclone - Pam Severity: WARNING, Max Winds 213kph/132mph, Gusts of 259kph/161mph http://…', chunks=['RT @disasteraware: #CYCLONE Alert-Update: Tropical Cyclone - Pam Severity: WARNING, Max Winds 213kph/132mph, Gusts of 259kph/161mph http://…'], id=None, images=None),
 Doc(meta={}, results={'language_detector': [('english', 1.0), ('other', 0.0)]}, uri="tweet_'501058011079933952'", text=

Filter for english tweets

In [29]:
# Filter for English tweets with confidence > 0.6
english_docs = [
    doc for doc in language_detection_results 
    if doc.results["language_detector"][0][0] == "english" and doc.results["language_detector"][0][1] > 0.6
    or doc.results["language_detector"][1][0] == "english" and doc.results["language_detector"][1][1] > 0.6
]

print(f"Number of English tweets: {len(english_docs)}")
print(f"Number of non-English tweets: {len(language_detection_results) - len(english_docs)}")


Number of English tweets: 100
Number of non-English tweets: 0


### Main Pipeline


Define the tasks

In [30]:
# Zero-shot taskswith default configuration
class Country(BaseModel):
    name: Literal["India", "California", "Pakistan", "Mexico", "Senegal", "Vanuatu", "Nepal", "Philippines", "Unknown"] = Field(description="The name of the country mentioned in the tweet if any. If not, return 'unknown'")

class CrisisType(BaseModel):
    crisis_type: Literal["Earthquake", "Flood", "Hurricane", "Typhoon", "Cyclone", "Ebola", "Unknown"] = Field(description="The type of crisis mentioned in the tweet")

crisis_classifier = tasks.Classification(
        task_id="crisis_classifier",
        labels=["related_to_crisis", "not_related_to_crisis"],
        model=model
    )

crisis_type_extractor = tasks.InformationExtraction(
        task_id="crisis_type_extractor",
        entity_type=CrisisType,
        model=model
    )

location_extractor = tasks.InformationExtraction(
        task_id="location_extractor",
        entity_type=Country,
        model=model
    )


/Users/angelo/Desktop/repo-sieves/.sieves-venv/lib/python3.12/site-packages/sieves/tasks/predictive/information_extraction/core.py:85: UserWarning: Entity type provided to task crisis_type_extractor isn't frozen, which means that entities can't be deduplicated. Modify entity_type to be frozen=True.
  warnings.warn(
/Users/angelo/Desktop/repo-sieves/.sieves-venv/lib/python3.12/site-packages/sieves/tasks/predictive/information_extraction/core.py:85: UserWarning: Entity type provided to task location_extractor isn't frozen, which means that entities can't be deduplicated. Modify entity_type to be frozen=True.
  warnings.warn(


Define Main Pipeline

In [31]:
pipe = Pipeline([
    crisis_classifier,
    crisis_type_extractor,
    location_extractor,
])

print("Pipeline created successfully!")
print(f"Number of tasks: {len(pipe.tasks)}")

Pipeline created successfully!
Number of tasks: 3


Run the pipeline

In [32]:
try:
    results = list(pipe(english_docs))
    
    # Display sample results
    for i, doc in enumerate(results):
        print(f"\\n--- Tweet {i+1} ---")
        print(f"Text: {doc.text}...")
        print(f"Crisis Classifier: {doc.results.get('crisis_classifier', 'N/A')}")
        print(f"Crisis Type Extraction: {doc.results.get('crisis_type_extractor')}")
        print(f"Locations Found: {doc.results.get('location_extractor', 'N/A')}")
        
except Exception as e:
    print(f"Error running pipeline: {e}")


Running pipeline: 100%|██████████| 100/100 [12:09<00:00,  7.29s/it]  

\n--- Tweet 1 ---
Text: RT @BuzzFeedStorm: Super Typhoon Hagupit strengthens with 178 mph max winds as storm tracks toward Philippines, JTWC reports http://t.co/ay…...
Crisis Classifier: [('related_to_crisis', 0.0), ('not_related_to_crisis', 0.0)]
Crisis Type Extraction: [CrisisType(crisis_type='Typhoon')]
Locations Found: [Country(name='Philippines')]
\n--- Tweet 2 ---
Text: RT @disasteraware: #CYCLONE Alert-Update: Tropical Cyclone - Pam Severity: WARNING, Max Winds 213kph/132mph, Gusts of 259kph/161mph http://…...
Crisis Classifier: [('related_to_crisis', 0.0), ('not_related_to_crisis', 0.0)]
Crisis Type Extraction: [CrisisType(crisis_type='Cyclone')]
Locations Found: [Country(name='Vanuatu')]
\n--- Tweet 3 ---
Text: @justinbieber i definitely accept if you are the one who warms me up afterwards? haha love you babe&lt;3↩️...
Crisis Classifier: [('related_to_crisis', 0.0), ('not_related_to_crisis', 0.0)]
Crisis Type Extraction: [CrisisType(crisis_type='Unknown')]
Locations Found: [Co

### Evaluating Results

Create a structured DataFrame with the results

In [33]:
evaluation_data = []
for i, doc in enumerate(results):
    # Get crisis classification
    crisis_result = doc.results.get('crisis_classifier', [])
    predicted_label = crisis_result[0][0] if crisis_result else 'unknown'
    predicted_conf = crisis_result[0][1] if crisis_result else 0.0
    
    # Get crisis type extraction
    crisis_type_result = doc.results.get('crisis_type_extractor', [])
    crisis_type = crisis_type_result[0].crisis_type if crisis_type_result else 'none_extracted'
    
    # Get location extraction
    location_result = doc.results.get('location_extractor', [])
    locations = [loc.name for loc in location_result] if location_result else []
    
    # Get gold labels from metadata
    gold_label = doc.metadata.get('gold_label', 'unknown')
    gold_crisis_type = doc.metadata.get('crisis_type', 'unknown')
    gold_location = doc.metadata.get('location', 'unknown')
    
    evaluation_data.append({
        'tweet_id': doc.metadata.get('tweet_id', ''),
        'text': doc.text[:100],
        'predicted_label': predicted_label,
        'predicted_confidence': predicted_conf,
        'gold_label': gold_label,
        'predicted_crisis_type': crisis_type,
        'gold_crisis_type': gold_crisis_type,
        'predicted_locations': ', '.join(locations) if locations else 'none',
        'gold_location': gold_location,
        'crisis_type_extracted': crisis_type != 'none_extracted',  # Track if extraction worked
        'location_extracted': any(loc != 'unknown' for loc in locations) if locations else False,  # Track if real locations found
        'location_match': any(gold_location.lower() in loc.lower() for loc in locations) if locations else False
    })

df_eval = pd.DataFrame(evaluation_data)

Analyze the obtained results

In [34]:
print("=" * 80)
print("PIPELINE PERFORMANCE")
print("=" * 80)

# 1. Crisis Classification Performance
df_eval['gold_binary'] = df_eval['gold_label'].apply(
    lambda x: 'not_related_to_crisis' if 'not_related' in x.lower() or 'irrelevant' in x.lower() 
    else 'related_to_crisis'
)
classification_accuracy = (df_eval['predicted_label'] == df_eval['gold_binary']).sum() / len(df_eval) * 100

print(f"\n1. CRISIS CLASSIFICATION")
print(f"   Accuracy: {classification_accuracy:.1f}%")
print(f"   Predicted: {sum(df_eval['predicted_label'] == 'related_to_crisis')} crisis, {sum(df_eval['predicted_label'] == 'not_related_to_crisis')} not crisis")

# 2. Crisis Type Extraction Performance  
crisis_type_success = df_eval['crisis_type_extracted'].sum()
crisis_type_total = len(df_eval)

print(f"\n2. CRISIS TYPE EXTRACTION")
print(f"   Extracted from: {crisis_type_success}/{crisis_type_total} tweets")
print(f"   Success rate: {crisis_type_success/crisis_type_total*100:.1f}%")
if crisis_type_success > 0:
    successful = df_eval[df_eval['crisis_type_extracted'] == True]
    types = successful['predicted_crisis_type'].value_counts()
    print(f"   Types found: {', '.join(types.index.tolist())}")

# 3. Location Extraction Performance
location_success = df_eval['location_extracted'].sum()
location_total = len(df_eval)

print(f"\n3. LOCATION EXTRACTION")
print(f"   Extracted from: {location_success}/{location_total} tweets")
print(f"   Success rate: {location_success/location_total*100:.1f}%")
if location_success > 0:
    successful_locs = df_eval[df_eval['location_extracted'] == True]['predicted_locations']
    all_locations = set()
    for locs in successful_locs:
        all_locations.update(locs.split(', '))
    print(f"   Locations found: {len(all_locations)} unique locations")

print("\n" + "=" * 80)


PIPELINE PERFORMANCE

1. CRISIS CLASSIFICATION
   Accuracy: 78.0%
   Predicted: 100 crisis, 0 not crisis

2. CRISIS TYPE EXTRACTION
   Extracted from: 90/100 tweets
   Success rate: 90.0%
   Types found: Earthquake, Hurricane, Typhoon, Unknown, Ebola, Cyclone, Flood

3. LOCATION EXTRACTION
   Extracted from: 99/100 tweets
   Success rate: 99.0%
   Locations found: 8 unique locations



Display the evaluation df

In [35]:
df_eval.head(10)

,tweet_id,text,predicted_label,predicted_confidence,gold_label,predicted_crisis_type,gold_crisis_type,predicted_locations,gold_location,crisis_type_extracted,location_extracted,location_match,gold_binary
0,'540375974136004611',RT @BuzzFeedStorm: Super Typhoon Hagupit stren...,related_to_crisis,0.0,other_useful_information,Typhoon,Typhoon,Philippines,Philippines,True,True,True,related_to_crisis
1,'575858180917342208',RT @disasteraware: #CYCLONE Alert-Update: Trop...,related_to_crisis,0.0,other_useful_information,Cyclone,Cyclone,Vanuatu,Vanuatu,True,True,True,related_to_crisis
2,'501058011079933952',@justinbieber i definitely accept if you are t...,related_to_crisis,0.0,not_related_or_irrelevant,Unknown,Floods,Unknown,India,True,True,False,not_related_to_crisis
3,'501265904185782272',RT @YaBoyBoober: I wanna be laid up is that to...,related_to_crisis,0.0,not_related_or_irrelevant,Unknown,Floods,Unknown,India,True,True,False,not_related_to_crisis
4,'497088306828828673',Ebola is an infectious disease caused by a vir...,related_to_crisis,0.0,disease_transmission,Ebola,Ebola,Unknown,Senegal,True,True,False,related_to_crisis
5,'505247380812201984',"RT @RAIDERS: ICYMI, Willie Brown presented a d...",related_to_crisis,0.0,donation_needs_or_offers_or_volunteering_services,Earthquake,Earthquake,California,California,True,True,True,related_to_crisis
6,'526368700077973505',@TheXFactor can i please have a #TXFSign from ...,related_to_crisis,0.0,not_related_or_irrelevant,none_extracted,Ebola,Unknown,Senegal,False,True,False,not_related_to_crisis
7,'504788244949897216',"RT @TheWineHub: ""#Winemakers R surveying the d...",related_to_crisis,0.0,other_useful_information,Earthquake,Earthquake,California,California,True,True,True,related_to_crisis
8,'592137589601267713',Learn about #Mountain #Ecosystems #EarthQuake ...,related_to_crisis,0.0,other_useful_information,Earthquake,Earthquake,Nepal,Nepal,True,True,True,related_to_crisis
9,'501511633978818560',"Hit Hard By #Ebola, #Liberia Now Has A Third T...",related_to_crisis,0.0,treatment,Ebola,Ebola,Unknown,Senegal,True,True,False,related_to_crisis


### Summary

This use case demonstrates the power of **Sieves** for common NLP tasks:

✅ **Language Detection** - Automatically filters non-English tweets  
✅ **Classification** - Zero-shot classification to identify crisis-related content  
✅ **Information Extraction** - Structured extraction of crisis types and locations

- [Sieves Documentation](https://sieves.ai/)
- [CrisisNLP Dataset](https://crisisnlp.qcri.org/lrec2016/lrec2016.html)
- [Sieves GitHub](https://github.com/mantisai/sieves)
